## Homework: Agents
----
In this homework, we will learn more about function calling, and we will also explore MCP - model-context protocol.
import random

known_weather_data = {
    'berlin': 20.0
}

### Preparation
---
First, we'll define a function that we will use when building our agent.

It will generate fake weather data:

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description
---
We want to use it as a tool for our agent, so we need to describe it

How should the description for this function look like? Fill in missing parts
```
get_weather_tool = {
    "type": "function",
    "name": "<TODO1>",
    "description": "<TODO2>",
    "parameters": {
        "type": "object",
        "properties": {
            "<TODO3>": {
                "type": "string",
                "description": "<TODO4>"
            }
        },
        "required": [TODO5],
        "additionalProperties": False
    }
}
```
What did you put in TODO3? **"string"**

Q2. Adding another tool
---
Let's add another tool - a function that can add weather data to our database:

In [2]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

Now let's write a description for it.

```
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set temperature for a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "cit to set the temperature of."
            },
            "weather": {
                "type": "float",
                "description": " new temperature"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}
```

## MCP
---
MCP stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:

A tool can export a list of functions it has
When we include the tool to our Agent, we just need to include the link to the MCP server

Q3. Install FastMCP
---
Let's install a library for MCP - [FastMCP](https://github.com/jlowin/fastmcp):

In [3]:
pip install fastmcp

Note: you may need to restart the kernel to use updated packages.


What's the version of FastMCP you installed?  **fastmcp-2.10.5**

## Q4. Simple MCP Server
---
A simple MCP server from the documentation looks like that:

In [4]:
# weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    await mcp.run_async(transport="http")

╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Demo 🚀                                             │
│    📦 Transport:       Streamable-HTTP                                     │
│    🔗 Server URL:      http://127.0.0.1:8000/mcp/                          │
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️  FastMCP version: 2.10.5                                              │
│    🤝 MCP version:     1.11.0                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯

[07/15/25 17:11:51] INFO     Starting MCP server 'Demo 🚀' with transport 'http' on                  ]8;id=289229;file:///usr/local/python/3.12.1/lib/python3.12/site-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=791766;file:///usr/local/python/3.12.1/lib/python3.12/site-packages/fastmcp/server/server.py#1448\1448]8;;\
                             http://127.0.0.1:8000/mcp/                                                            

INFO:     Started server process [4445]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
ERROR:    Cancel 0 running task(s), timeout graceful shutdown exceeded
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4445]


In our case, we need to write docstrings for our functions.

Let's ask ChatGPT for help:

In [12]:
import asyncio

mcp = FastMCP("Weather Server 🌦️")

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

@mcp.tool
def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

# ✅ Run the MCP server in the background
async def start_server():
    await mcp.run_async(transport="http", port=8000)

# Start in background so notebook remains usable
asyncio.create_task(start_server())

<Task pending name='Task-21' coro=<start_server() running at /tmp/ipykernel_4445/4187873434.py:40>>



╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Weather Server 🌦️                                    │
│    📦 Transport:       Streamable-HTTP                                     │
│    🔗 Server URL:      http://127.0.0.1:8000/mcp

[07/15/25 17:26:32] INFO     Starting MCP server 'Weather Server 🌦️' with transport 'http' on         ]8;id=97076;file:///usr/local/python/3.12.1/lib/python3.12/site-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=356165;file:///usr/local/python/3.12.1/lib/python3.12/site-packages/fastmcp/server/server.py#1448\1448]8;;\
                             http://127.0.0.1:8000/mcp/                                                            

INFO:     Started server process [4445]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Let's change the example for our case and run it

What do you see in the output?

Look for a string that matches this template:
```
Starting MCP server 'Demo 🚀' with transport '<TODO>'
```
What do you have instead of ```<TODO>```?   **'http'**

## Q5. Protocol
---
There are different ways to communicate with an MCP server. Ours is currently running using standart input/output, which means that the client write something to stdin and read the answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:

First, we send an initialization request -- this way, we register our client with the server:

```{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}```

In [13]:
import requests
import json

MCP_URL = "http://127.0.0.1:8000/mcp/"  # adjust if you specified a different port
headers = {"Content-Type": "application/json"}

init_payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": "2024-11-05",
        "capabilities": {
            "roots": {"listChanged": True},
            "sampling": {}
        },
        "clientInfo": {
            "name": "test-client",
            "version": "1.0.0"
        }
    }
}

response = requests.post(MCP_URL, json=init_payload, headers=headers)
print(response.json())

KeyboardInterrupt: 

In [14]:
curl http://localhost:8000

SyntaxError: invalid syntax (2836940682.py, line 1)